In [1]:
#!pip install --upgrade keras-cv tensorflow
#!pip install --upgrade keras
!pip install keras_tuner
import tensorflow as tf
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import keras
import datetime
%load_ext tensorboard
import keras_tuner as kt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
"""from google.colab import drive
drive.mount('/content/gdrive')"""

"from google.colab import drive\ndrive.mount('/content/gdrive')"

In [2]:
cifar10 = tf.keras.datasets.cifar10.load_data()
(X_train_full, y_train_full), (X_test, y_test) = cifar10


170498071/170498071 [==============================] - 19s 0us/step


In [3]:
X_train_full = tf.image.rgb_to_grayscale(X_train_full)
X_test = tf.image.rgb_to_grayscale(X_test)

In [4]:
X_train_full.shape

TensorShape([50000, 32, 32, 1])

In [5]:
"""to_grayscale = keras.layers.preprocessing.Grayscale()
X_train_full = to_grayscale(X_train_full)
X_test = to_grayscale(X_test)"""

'to_grayscale = keras.layers.preprocessing.Grayscale()\nX_train_full = to_grayscale(X_train_full)\nX_test = to_grayscale(X_test)'

In [6]:
X_train, y_train = X_train_full[:-10000], y_train_full[:-10000]
X_valid, y_valid = X_train_full[-10000:], y_train_full[-10000:]

In [7]:
print(X_train[1][0][1])

tf.Tensor([133], shape=(1,), dtype=uint8)


In [8]:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
class_names[int(y_train[5])]


<ipython-input-8-f94cb98b12e9>:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  class_names[int(y_train[5])]


'automobile'

In [9]:



model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = [32, 32, 1]),
    tf.keras.layers.Dense(800, kernel_initializer = 'random_normal',bias_initializer = 'zeros' ,activation = "relu"),
    tf.keras.layers.Dense(700, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(600, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(600, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(600, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(400, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(10, activation = "softmax")
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 800)               820000    
                                                                 
 dense_1 (Dense)             (None, 700)               560700    
                                                                 
 dense_2 (Dense)             (None, 600)               420600    
                                                                 
 dense_3 (Dense)             (None, 600)               360600    
                                                                 
 dense_4 (Dense)             (None, 600)               360600    
                                                                 
 dense_5 (Dense)             (None, 400)               2

In [10]:
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = "sgd",
              metrics = "accuracy")

In [11]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
checkpoints = tf.keras.callbacks.ModelCheckpoint('./checkpoint.model.keras')
history = model.fit(X_train, y_train, epochs = 30, validation_data = (X_valid, y_valid), callbacks=[tensorboard_callback, checkpoints])

Epoch 1/30
1250/1250 [==============================] - 22s 14ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 2/30
1250/1250 [==============================] - 7s 6ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 3/30
1250/1250 [==============================] - 7s 6ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 4/30
1250/1250 [==============================] - 7s 6ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 5/30
1250/1250 [==============================] - 8s 7ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 6/30
1250/1250 [==============================] - 8s 6ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 7/30
1250/1250 [==============================] - 7s 6ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 8/30
1250/1250 [==========

In [12]:
weights = model.get_layer("dense_1").get_weights()

In [13]:
weights[1].shape

(700,)

In [14]:
# Load the TensorBoard notebook extension


In [15]:
def build_model(hp):
  n_hidden = hp.Int("n_hidden", min_value = 0, max_value = 10, default = 2)
  n_neurons = hp.Int("n_neurons", min_value = 10, max_value = 500, default = 3)
  learning_rate = hp.Float("learning_rate", min_value = 1e-4, max_value = 1e-2, sampling = "log")
  optimizer = hp.Choice("optimizer", values = ["sgd", "adam"])
  if optimizer == "sgd":
    optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)
  else:
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape = [32,32,1]))
  for i in range(n_hidden):
    model.add(tf.keras.layers.Dense(n_neurons, activation = "relu"))
  model.add(tf.keras.layers.Dense(10, activation = "softmax"))
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
  return model



In [16]:
random_search_tuner = kt.RandomSearch(build_model,
                                  objective = "accuracy",
                                  max_trials = 100,
                                  seed = 42,
                                  directory = "cifarproject",
                                  project_name = "random_search_runs")
random_search_tuner.search(X_train, y_train, epochs = 5, validation_data = (X_valid, y_valid))

Trial 100 Complete [00h 00m 42s]
accuracy: 0.2178249955177307

Best accuracy So Far: 0.3695499897003174
Total elapsed time: 01h 00m 58s


In [ ]:
!rm -rf cifarproject/

In [20]:
top5_trials = random_search_tuner.oracle.get_best_trials(num_trials = 5)
for _ in top5_trials:
  print(_.summary())

Trial 019 summary
Hyperparameters:
n_hidden: 7
n_neurons: 464
learning_rate: 0.0010712131812413617
optimizer: sgd
Score: 0.3695499897003174
None
Trial 017 summary
Hyperparameters:
n_hidden: 10
n_neurons: 416
learning_rate: 0.0009172617171127613
optimizer: sgd
Score: 0.3644999861717224
None
Trial 060 summary
Hyperparameters:
n_hidden: 7
n_neurons: 365
learning_rate: 0.00011631756169915276
optimizer: adam
Score: 0.35534998774528503
None
Trial 034 summary
Hyperparameters:
n_hidden: 10
n_neurons: 452
learning_rate: 0.00019430304760198347
optimizer: adam
Score: 0.3549000024795532
None
Trial 032 summary
Hyperparameters:
n_hidden: 8
n_neurons: 393
learning_rate: 0.0005070370705614628
optimizer: sgd
Score: 0.35397499799728394
None


In [29]:
best_trial = tf.keras.Sequential()
best_trial.add(tf.keras.layers.Flatten(input_shape = [32,32,1]))
for _ in range(7):
  best_trial.add(tf.keras.layers.Dense(464, activation = "relu"))
best_trial.add(tf.keras.layers.Dense(10, activation = "softmax"))

best_trial.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 1024)              0         
                                                                 
 dense_11 (Dense)            (None, 464)               475600    
                                                                 
 dense_12 (Dense)            (None, 464)               215760    
                                                                 
 dense_13 (Dense)            (None, 464)               215760    
                                                                 
 dense_14 (Dense)            (None, 464)               215760    
                                                                 
 dense_15 (Dense)            (None, 464)               215760    
                                                                 
 dense_16 (Dense)            (None, 464)              

In [30]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.0010712131812413617)
best_trial.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

In [31]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience = 10,
                                                     restore_best_weights = True)
checkpoints_b = tf.keras.callbacks.ModelCheckpoint('./checkpoint_b.model.keras')
history_b = best_trial.fit(X_train, y_train, epochs = 40, validation_data = (X_valid, y_valid), callbacks=[early_stopping_cb, checkpoints])

Epoch 1/40
1250/1250 [==============================] - 8s 6ms/step - loss: 2.2840 - accuracy: 0.2416 - val_loss: 1.9890 - val_accuracy: 0.2827
Epoch 2/40
1250/1250 [==============================] - 6s 5ms/step - loss: 1.9406 - accuracy: 0.3044 - val_loss: 1.9404 - val_accuracy: 0.2994
Epoch 3/40
1250/1250 [==============================] - 7s 6ms/step - loss: 1.8696 - accuracy: 0.3329 - val_loss: 1.8945 - val_accuracy: 0.3223
Epoch 4/40
1250/1250 [==============================] - 6s 5ms/step - loss: 1.8153 - accuracy: 0.3533 - val_loss: 1.8621 - val_accuracy: 0.3415
Epoch 5/40
1250/1250 [==============================] - 7s 5ms/step - loss: 1.7731 - accuracy: 0.3678 - val_loss: 1.8373 - val_accuracy: 0.3430
Epoch 6/40
1250/1250 [==============================] - 6s 5ms/step - loss: 1.7354 - accuracy: 0.3807 - val_loss: 1.8400 - val_accuracy: 0.3406
Epoch 7/40
1250/1250 [==============================] - 6s 5ms/step - loss: 1.7019 - accuracy: 0.3938 - val_loss: 1.8041 - val_accuracy:

In [32]:
best_trial.evaluate(X_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 1.7676 - accuracy: 0.3786


[1.7676118612289429, 0.37860000133514404]